In [1]:
import os
import pandas as pd

from Bio import AlignIO

1. Program versions:
- `clustalo` 1.2.4
- `clustalw` 2.1
- `mafft` v7.520
- `muscle` 5.1.linux64
- `kalign` 2.04
- `t_coffee` Version_12.00.7fb08c2
- `prank` v.170427


2. Run 6 algos (clustalw, muscle, mafft, kalign, t_coffee, prank) on `.fasta` file with 10 sequences

In [2]:
time_dct = {}
time_dct["clustalw"] = %timeit -n1 -r1 -o !clustalw -INFILE=SUP35_10seqs.fa -OUTPUT=FASTA -OUTFILE=02_SUP35_10seqs_clustalw.fa > logs/logs_clustalw.txt
time_dct["muscle"] = %timeit -n1 -r1 -o !muscle -align SUP35_10seqs.fa -output 02_SUP35_10seqs_muscle.fa 2> logs/logs_muscle_std2.txt 1> logs/logs_muscle_std1.txt
time_dct["mafft"] = %timeit -n1 -r1 -o !mafft --auto SUP35_10seqs.fa >02_SUP35_10seqs_mafft.fa 1> logs/logs_mafft_std1.txt 2> logs/logs_mafft_std2.txt
time_dct["kalign"] = %timeit -n1 -r1 -o !kalign <SUP35_10seqs.fa >02_SUP35_10seqs_kalign.fa 2> logs/logs_kalign.txt
time_dct["tcoffee"] = %timeit -n1 -r1 -o !t_coffee -infile=SUP35_10seqs.fa -outfile=02_SUP35_10seqs_tcoffee.fa -output=fasta_aln 2> logs/logs_t_coffee_std2.txt 1> logs/logs_t_coffee_std1.txt
time_dct["prank"] = %timeit -n1 -r1 -o !prank -d=SUP35_10seqs.fa -o=02_SUP35_10seqs_prank.fa -codon > logs/logs_prank.txt # codon for codon alignment

for k in time_dct:
    time_dct[k] = round(time_dct[k].average, 2)

2.68 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
6.06 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.53 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
343 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
24.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1min 16s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [3]:
time_dct

{'clustalw': 2.68,
 'muscle': 6.06,
 'mafft': 2.53,
 'kalign': 0.34,
 'tcoffee': 24.78,
 'prank': 76.88}

3. Get comparative table for every aln algo with time and aln length

Simple func to get aln length from fasta

In [4]:
def get_aln_len(path: str) -> int:
    with open(path) as f:
        aln_len = 0
        for line in f:
            if line.strip().startswith(">") and aln_len != 0:
                break
            else:
                aln_len += len(line.strip())
    return aln_len

In [5]:
length_dct = {}

for file in os.listdir():
    if file.startswith("02_SUP35_10seqs"):
        length_dct[file.split("_")[-1].split(".")[0]] = get_aln_len(file)

In [6]:
length_dct

{'clustalw': 2177,
 'muscle': 2368,
 'tcoffee': 2229,
 'prank': 2453,
 'mafft': 2185,
 'kalign': 2174}

In [8]:
pd.DataFrame({"time": time_dct, "length": length_dct}).sort_values(by=["time", "length"])

,time,length
kalign,0.34,2174
mafft,2.53,2185
clustalw,2.68,2177
muscle,6.06,2368
tcoffee,24.78,2229
prank,76.88,2453


In terms of time the best algo is kalign, as well, as in aln length. Smaller length usually means less gaps, but one should consider biological sense behind any alignment.

4. Let's tale a look at `SUP35_10seqs_strange_aln.fa`

In [11]:
!cat SUP35_10seqs_strange_aln.fa

>SUP35_Kla_AB039749
ATGTCAG---------------------------ACCAACAAAATCAAGACCAAGGGCAAGGCCAAGGTTACAATCAGTATAACCAATATGGCCAGTACAACCAG----TACTACAACCAACAGGGCTATCAAGGCTACAACGGCCAACAAGGTGCTCCTCAAGGCTACCAAGCATATCAAGCTTATGGACAGCAGCCTCAAGG---------------------AGCCTACCAGGGCTAC---AACCCTC--AACAAGCTCAAG-----GCT---ACCAACCTTACCAGGGCT---ACAATGCTCAGCAA------CAAGGTTACAAC-----------GCTCAG------------------------CAAGGCGGTCA-CAACAATAACTACAATAAAAA--------TTATAATAATAA---AAACAGTTACAATAACTATAATAAGCA-GGGTTATCAAGGTGCTCAAGGATATAATGCACAACAGCCAACCGGTTACGCTGCTCCAGCACAGTCTTCATCCCAGGGTATGACT-----TTGAAAGATTTCCAAAA-------CCAAC-AAGGCAGTACTAATGCAGCCAAGCCAAAGCCTAAGTTAAAGTTGGCCTCTAGCTCTGGTATTAAGTTAGTAGGTG---CCAAGAAACCTGTAGCA------CCCAAAACTGAGAAAA-----------------CTGATGAATC--CAAGGAAGCAACTAAA------------------------------------------------ACTACCGACGACAACGAAGAAGCACAATCTGAATTGCCCAAAATTGATG-ATTTGAAAATCTCAGAGGCTGAAAAACCAAAAACTAAGGAGAATACCCCATCTGCTGATGATACTTCCTCAGAGAAGACTACCAGCGCTAAGGCAGACACATCTACAGGAGGAGCGAACTCCGTGGAT------GCTCTAATCAAG

Some long sequences here definetely ruin whole alignment. We could try to drop them or reverse or make them shorter (some of them align well at the beginning, others at the end)

5. Run same aligners, but on longer sequences

In [54]:
# time_dct = {}
# time_dct["clustalw"] = %timeit -n1 -r1 -o !clustalw -INFILE=SUP35_250seqs.fa -OUTPUT=FASTA -OUTFILE=05_SUP35_250seqs_clustalw.fa > logs/logs_clustalw.txt
# time_dct["muscle"] = %timeit -n1 -r1 -o !muscle -align SUP35_250seqs.fa -output 05_SUP35_250seqs_muscle.fa 2> logs/logs_muscle_std2.txt 1> logs/logs_muscle_std1.txt
# time_dct["mafft"] = %timeit -n1 -r1 -o !mafft --auto SUP35_250seqs.fa > 05_SUP35_250seqs_mafft.fa 1> logs/logs_mafft_std1.txt 2> logs/logs_mafft_std2.txt
# time_dct["kalign"] = %timeit -n1 -r1 -o !kalign <SUP35_250seqs.fa> 05_SUP35_250seqs_kalign.fa 2> logs/logs_kalign.txt
# time_dct["tcoffee"] = %timeit -n1 -r1 -o !t_coffee -infile=SUP35_250seqs.fa -outfile=05_SUP35_250seqs_tcoffee.fa -output=fasta_aln 2> logs/logs_t_coffee_std2.txt 1> logs/logs_t_coffee_std1.txt
# time_dct["prank"] = %timeit -n1 -r1 -o !prank -d=SUP35_250seqs.fa -o=05_SUP35_250seqs_prank.fa > logs/logs_prank.txt

# for k in time_dct:
#     time_dct[k] = round(time_dct[k].average, 2)

In [41]:
time_dct = {
    "clustalw": 1361.64,
    "muscle": "too long :(", # option -super5 can be used, but i didnt try it
    "mafft": 20.96,
    "kalign": 12.63,
    "tcoffee": 26261.42,
    "prank": 185.78,
}

6. Comparative table (time and aln length)

In [43]:
length_dct = {}

for file in os.listdir():
    if file.startswith("05_SUP35_250seqs"):
        length_dct[file.split("_")[-1].split(".")[0]] = get_aln_len(file)

In [44]:
length_dct

{'mafft': 2341,
 'tcoffee': 2269,
 'clustalw': 2207,
 'prank': 2480,
 'kalign': 2233,
 'muscle': 0}

In [45]:
pd.DataFrame({"time": time_dct, "length": length_dct}).sort_values(
    by=["time", "length"]
)

,time,length
kalign,12.63,2233
mafft,20.96,2341
prank,185.78,2480
clustalw,1361.64,2207
tcoffee,26261.42,2269
muscle,too long :(,0


Shorter alignment with best time was obtained with kalign again. But we might want to consider biological sense too.

7. Getting aminoacid sequences from nucleotide

In [50]:
!transeq -sequence SUP35_10seqs.fa -outseq SUP35_10seqs.t.faa
# !cat SUP35_10seqs.t.faa


Translate nucleic acid sequences


In [53]:
!getorf -sequence SUP35_10seqs.fa -outseq SUP35_10seqs.g.faa -noreverse -minsize 500
# !cat SUP35_10seqs.g.faa

Find and extract open reading frames (ORFs)


We need to consider ORFs. Automatic ORF finders might be too "mechanical", if every start-codon is ORF opening for them.

8. Aligning protein sequences

In [94]:
time_dct = {}
time_dct["clustalw"] = %timeit -n1 -r1 -o !clustalw -INFILE=SUP35_10seqs.g.faa -OUTFILE=08_SUP35_10seqs_clustalw.faa -OUTPUT=FASTA -TYPE=protein   
time_dct["clustalo"] = %timeit -n1 -r1 -o !clustalo --infile=SUP35_10seqs.g.faa --outfile=08_SUP35_10seqs_clustalo.faa # [--verbose]   
time_dct["muscle"] = %timeit -n1 -r1 -o !muscle -align SUP35_10seqs.g.faa -output 08_SUP35_10seqs_muscle.faa  
time_dct["mafft"] = %timeit -n1 -r1 -o !mafft --auto SUP35_10seqs.g.faa >08_SUP35_10seqs_mafft.fa  
time_dct["kalign"] = %timeit -n1 -r1 -o !kalign <SUP35_10seqs.g.faa> 08_SUP35_10seqs_kalign.faa  
# time_dct["tcoffee"] = %timeit -n1 -r1 -o !export MAX_N_PID_4_TCOFFEE=400000 && echo $MAX_N_PID_4_TCOFFEE && t_coffee -infile=SUP35_10seqs.g.faa -outfile=08_SUP35_10seqs_tcoffee.faa -output=fasta_aln  
# couldn't manage to fix it :(
time_dct["prank"] = %timeit -n1 -r1 -o !prank -d=SUP35_10seqs.g.faa -o=08_SUP35_10seqs_prank.faa  

for k in time_dct:
    time_dct[k] = round(time_dct[k].average, 2)




 CLUSTAL 2.1 Multiple Sequence Alignments


Sequence type explicitly set to Protein
Sequence format is Pearson
Sequence 1: SUP35_Kla_AB039749_1                       700 aa
Sequence 2: SUP35_Agos_ATCC_10895_NM_211584_1          691 aa
Sequence 3: SUP35_Scer_74-D694_GCA_001578265.1_1       685 aa
Sequence 4: SUP35_Spar_A12_Liti_1                      681 aa
Sequence 5: SUP35_Smik_IFO1815T_30_1                   681 aa
Sequence 6: SUP35_Skud_IFO1802T_36_1                   678 aa
Sequence 7: SUP35_Sbou_unique28_CM003560_1             685 aa
Sequence 8: SUP35_Scer_beer078_CM005938_1              666 aa
Sequence 9: SUP35_Sarb_H-6_chrXIII_CM001575_1          680 aa
Sequence 10: SUP35_Seub_CBS12357_chr_II_IV_DF968535_1   674 aa
Start of Pairwise alignments
Aligning...

Sequences (1:2) Aligned. Score:  71
Sequences (1:3) Aligned. Score:  69
Sequences (1:4) Aligned. Score:  69
Sequences (1:5) Aligned. Score:  68
Sequences (1:6) Aligned. Score:  68
Sequences (1:7) Aligned. Score:  69
Sequenc

In [95]:
time_dct

{'clustalw': 0.41,
 'clustalo': 0.12,
 'muscle': 0.75,
 'mafft': 0.47,
 'kalign': 0.14,
 'prank': 7.53}

9. Comparative table for protein aln

In [96]:
length_dct = {}

for file in os.listdir():
    if file.startswith("08_SUP35_10seqs"):
        length_dct[file.split("_")[-1].split(".")[0]] = get_aln_len(file)

In [97]:
length_dct

{'clustalo': 789,
 'kalign': 752,
 'muscle': 807,
 'prank': 823,
 'mafft': 791,
 'clustalw': 756}

9. Comparative table for proteins

In [98]:
pd.DataFrame({"time": time_dct, "length": length_dct}).sort_values(
    by=["time", "length"]
)

,time,length
clustalo,0.12,789
kalign,0.14,752
clustalw,0.41,756
mafft,0.47,791
muscle,0.75,807
prank,7.53,823


10. Add new aligned sequences to existing aln with mafft

In [116]:
#!muscle -align SUP35_2addseqs.fa -output 10_SUP35_2addseqs_muscle.fa
# !muscle -profile -in1 05_SUP35_250seqs_muscle.fa -in2 SUP35_2addseqs.fa -out 10_SUP35_252seqs_muscle.fa
# unfortunately muscle v5 doesnt seem to have this option anymore (or i couldnt find it)
!mafft --auto SUP35_2addseqs.fa > 10_SUP35_2addseqs_mafft.fa
!mafft --add 10_SUP35_2addseqs_mafft.fa --reorder 05_SUP35_250seqs_mafft.fa > 10_SUP35_252seqs_mafft.fa

outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.520
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
0 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
    0 / 2
done.

Progressive alignment ... 
STEP     1 /1 
done.
tbfast (nuc) Version 7.520
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
1 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.
generating a scoring matrix for nucleotide (dist=200) ... done

    0 / 2
Segment   1/  1    1-2059
done 001-001-1  identical.   
dvtdit

11. Extracting Parapallasea 18S sequences and saving to file

In [122]:
!esearch -db nucleotide -query "Parapallasea 18S" | efetch -format fasta > 11_Parapallasea_18.fa


In [123]:
!mafft --auto 11_Parapallasea_18.fa > 11_Parapallasea_18_mafft.fa

outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
All-to-all alignment.
tbfast-pair (nuc) Version 7.520
alg=L, model=DNA200 (2), 2.00 (6.00), -0.10 (-0.30), noshift, amax=0.0
0 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
    0 / 4
done.

Progressive alignment ... 
STEP     3 /3 
done.
tbfast (nuc) Version 7.520
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
1 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 0
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.
generating a scoring matrix for nucleotide (dist=200) ... done

    0 / 4
Segment   1/  1    1-2270
STEP 002-002-1  identical.    identi

The resulting aligment has 3  well-aligned sequences, another aligns to the end. I suppose it's OK for rRNA genes to be that clustered

12. Making custom db for blast

In [126]:
!makeblastdb -in Ommatogammarus_flavus_transcriptome_assembly.fa -dbtype nucl -parse_seqids




Building a new DB, current time: 03/08/2024 16:27:13
New DB name:   /home/alex/dev/phylo/hw_3/Ommatogammarus_flavus_transcriptome_assembly.fa
New DB title:  Ommatogammarus_flavus_transcriptome_assembly.fa
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 100 sequences in 0.00353789 seconds.




In [127]:
!cat Acanthogammarus_victorii_COI.faa

>Acanthogammarus_victorii_COI
MIKHWLFSTNHKDIGTLYFILGAWAGMVGTSMSVIIRSELSAPGNLIGDDQLYNVMVTSHAFVMIFFMVMPIMIGGFGNWLVPLMLGSPDMAFPRMNNMSFWLLPPSLTLLITSGLVESGVGTGWTVYPPLSGTTAHSGSAVDLAIFSLHLAGASSILGAVNFISTIANMRSPSMALDQMPLFVWAILITTVLLLLSLPVLAGAITMLLTDRNLNTSFFDPGGGGDPILYQHLFWFFGHPEVYILILPAFGMVSHIVSQESGKKETFGSLGMIYAMLAIGFLGFIVWAHHMFTVGMDVDTRAYFTSATMIIAVPTGIKVFSWLGTLQGGSIKFTPALIWSLGFIFLFSIGGLTGVMLANSSIDIVLHDTYYVVAHFHYVLSMGAVFGIFAGFAYWFPLFTGVTANPVLAKAHFYAMFTGVNLTFFPQHFLGLTGMPRRYSDYPDFFTAWNTMSSVGSYISVVGMVLFVLMVMEAFVAKRPALFSLSLASALEWQHPYPPAGHSYSSTPILAS

For mitochondria has sligtly different genecode, we need to specify it via `-db_gencode 5` option

In [137]:
!tblastn -query Acanthogammarus_victorii_COI.faa -db Ommatogammarus_flavus_transcriptome_assembly.fa -outfmt 6 -db_gencode 5 > Acanthogammarus_victorii_COI_blast_output_6.txt


In [138]:
pd.read_csv(
    "Acanthogammarus_victorii_COI_blast_output_6.txt",
    sep="\t",
    header=None,
    names=[
        "qseqid",
        "sseqid",
        "pident",
        "length",
        "mismatch",
        "gapopen",
        "qstart",
        "qend",
        "sstart",
        "send",
        "evalue",
        "bitscore",
    ],
)

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,Acanthogammarus_victorii_COI,TRINITY_DN8878_c0_g1_i2,89.621,501,52,0,9,509,3,1505,0.0,781.0
1,Acanthogammarus_victorii_COI,TRINITY_DN58613_c0_g1_i1,50.000,20,9,1,206,225,32,88,6.3,20.8


Best match to separate file

In [139]:
!blastdbcmd -db Ommatogammarus_flavus_transcriptome_assembly.fa -entry TRINITY_DN8878_c0_g1_i2 -out Ommatogammarus_flavus_COI.fa 

In [140]:
!cat Ommatogammarus_flavus_COI.fa

>TRINITY_DN8878_c0_g1_i2 len=1505
CGACCAACCACAAAGATATTGGCACTCTTTATTTTATGCTAGGGCTCTGGTCTGGGTTAGTCGGAACCTCCATAAGACTT
ATCCTCCGCTCAGAACTTAGTGCGCCGGGTAGCCTGATTGGTGATGATCAACTGTATAACGTAATGGTAACCTCCCATGC
TTTTATTATAATTTTTTTTATAGTTATGCCTATCATAATTGGCGGGTTTGGTAACTGGCTGCTTCCTTTAATACTAGGTA
GACCTGATATAGCCTTCCCTCGAATAAACAACATGAGCTTTTGACTACTACCTCCTTCCCTTACACTTCTTATATCTAGA
AGCTTAGTAGAAAGAGGAGTCGGCACAGGTTGAACTGTCTACCCTCCTTTATCTGGGTCTACAGCCCATAGAGGTAGCGC
TGTAGATTTGGCTATTTTCTCACTTCATTTAGCCGGAGCTTCCTCTATCTTAGGGGCTGTAAATTTTATTTCTACCGCCA
TTAATATGCGAGCGCCTGGGATAAAATTAGACCAAATGCCTTTATTCGTCTGAGCTATTATTATTACTACCGTCCTCCTA
GTCTTATCCCTACCAGTCCTAGCTGGGGCCATTACGATACTACTTACAGACCGTAACATAAATACCTCTTTTTTTGACCC
TAGTGGGGGGGGTGACCCTATCCTATACCAACACTTATTTTGATTTTTTGGGCACCCAGAGGTGTATATTTTAATCCTGC
CTGCATTTGGCATAATCTCTCATATTGTTAGACAGGAGTCCGGTAAAAAAGAAACATTTGGCCCCCTAGGGATAATTTAT
GCTATATTAGCTATTGGGTTCCTCGGATTTATTGTGTGAGCCCATCATATGTTTACAGTCGGTATGGATGTAGATACCCG
AGCCTATTTTACATCAGCTACAATAATTATTGCAGTCCCCACCGGCATCAAAGTATTTAGGTGACTAGGTACTCT